In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/MyDrive/Colab Notebooks")

Mounted at /content/drive


In [ ]:
!pip install -q transformers
import transformers 
from transformers import GPT2Model, GPT2Tokenizer 
from transformers import BartModel, BartTokenizer 
from transformers import BertModel, BertTokenizer 

import torch
import numpy as np
import matplotlib.pyplot as plt
from torch import nn
import torch.nn.utils.prune as prune

import copy
import json
from itertools import chain

# Models

* ~GPT2-XL~ GPT2-medium
* ~Pangu $\alpha$ 2.6B: [Intro from Huawei](https://www.huaweicloud.com/product/modelarts/pangu.html)~
* ~erlangshen 1.3B: [Intro from IDEA](https://huggingface.co/IDEA-CCNL/Erlangshen-MegatronBert-1.3B)~

GPT2-XL works well for benchmarks testing. But it's too big to store on the disk on argonne's personal directory.So we use GPT-2 medium instead.

Structure of Chinese language models are really weird.  They're not well supported by huggingface and `transformers` API. Too many bugs. Instead, we use 
* bart-large
* bert-large 

# Visualization & Sparsification


In [ ]:
def get_params_plot(model, model_name, sparsity=0):
  # get params dist.
  fig, ax = plt.subplots(figsize=(12,8))
  total_param = torch.cat([params.flatten() for params in model.parameters()]).detach().numpy()
  hist, bins = np.histogram(total_param, bins = np.linspace(-1,1,201)) 
  ax.plot(bins[:-1],hist) 
  plt.xlabel("Weight values")
  plt.ylabel("Log Count")
  plt.yscale('log')
  plt.title("distribution of all weights(model:{}, sparsity:{}%)".format(model_name, sparsity*100), fontsize=25)
  plt.savefig('{}/figs/vis_all_params_{}_sparsity_{}%.png'.format(os.getcwd(), model_name, sparsity*100))
  
  # get params dist. in each layer
  for elem in model.named_children():
    if elem[0] in ['h', 'encoder', 'decoder']: 
      fig, ax = plt.subplots(figsize=(12,8))
      for idx, child in enumerate(elem[1].named_children()):
        layer_param = torch.cat([params.flatten() for params in child[1].parameters()]).detach().numpy()
        hist, bins = np.histogram(layer_param, bins = np.linspace(-1,1,201)) 
        ax.plot(bins[:-1],hist,label="{} hidden layer".format(idx)) 
      plt.xlabel("Weight values")
      plt.ylabel("Log Count")
      plt.yscale('log')
      plt.legend(loc="upper left", bbox_to_anchor=(1, 1))
      plt.title("distribution of weights across layers(model:{}, sparsity:{}%)".format(model_name, sparsity*100), fontsize=25)
      plt.savefig('{}/figs/vis_layers_params_{}_sparsity_{}%.png'.format(os.getcwd(), model_name, sparsity*100))

In [ ]:
def prune_model(model, model_name, prune_portion):
  '''
  using global pruning to prune gpt2, bart, and bert. 
  '''
  def print_sparsity(module_lst, prune_flag):

    zero_params_count = sum([torch.sum(module[0].weight == 0) for module in module_lst])
    total_params_count = sum([module[0].weight.nelement() for module in module_lst])

    print(
        "Global sparsity of model: {} in hidden layers {} pruning: {:.2f}%".format(model_name, prune_flag,
                                                    100. * float(zero_params_count)
                                                        / float(total_params_count)
                                                    )
        )
    return None
    
  if model_name == 'BART-large':
    modules = chain(model.encoder.modules(), model.decoder.modules())
  elif model_name == 'BERT-large':
    modules = model.encoder.modules()
  else:
    modules = model.h.modules()

  module_lst = []
  for module in modules:
    if hasattr(module, 'weight'):
      module_lst.append((module, 'weight'))

  print_sparsity(module_lst, prune_flag = 'before')
  prune.global_unstructured(module_lst, 
                            pruning_method=prune.L1Unstructured, 
                            amount=prune_portion)
  for module, attr in module_lst: 
    prune.remove(module, attr)
  print_sparsity(module_lst, prune_flag = 'after')
  return model

In [ ]:
# Method from https://discuss.pytorch.org/t/finding-model-size/130275/5
def get_model_size(model):
  param_size = 0
  for param in model.parameters():
      param_size += param.nelement() * param.element_size()
  buffer_size = 0
  for buffer in model.buffers():
      buffer_size += buffer.nelement() * buffer.element_size()

  size_all_mb = (param_size + buffer_size) / 1024**2
  print('model size: {:.3f}MB'.format(size_all_mb))
  return size_all_mb

In [ ]:
model_names = ['GPT2-xl', 'BART-large', 'BERT-large']
sparsity = [0, 0.1, 0.5, 0.9, 0.95, 0.99] 
model_size_dic = dict()

print("Start sparsifying...")
for model_name in model_names:
  if model_name == 'GPT2-xl':
    model = GPT2Model.from_pretrained('gpt2-xl')
    # model = GPT2Model.from_pretrained('gpt2')
  elif model_name == 'BART-large':
    model = BartModel.from_pretrained("facebook/bart-large")
    # model = BartModel.from_pretrained("facebook/bart-base")
  else:
    # model = BertModel.from_pretrained("bert-base-cased")
    model = BertModel.from_pretrained("bert-large-cased")

  for prune_proportion in sparsity:
    target_model = copy.deepcopy(model)
    if prune_proportion != 0:
      target_model = prune_model(target_model, model_name, prune_proportion)
    model_size_dic[f'{model_name}_sparsity_{prune_proportion*100}%'] = get_model_size(target_model)
    get_params_plot(target_model, model_name, prune_proportion)
    target_model.save_pretrained("{}/models/{}_sparsity_{}".format(os.getcwd(), 
                                                            model_name,
                                                            prune_proportion*100,
                                                            ))
    print("model {} with sparsity {} saved".format(model_name, prune_proportion * 100))
    
print("Sparsification completed!")


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
dir_open = open(f'{os.getcwd()}/model_size.json', "w")
json.dump(model_size_dic, dir_open, indent = 4)

# Evaluation on Benchmarks

GLUE(qnli) and CLM(using wikitext)

~For pangu, erlangshen: CLUE (tnews and cmnli)~

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install -q transformers
import transformers 
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM 
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

import torch
import numpy as np
import pandas as pd 

!pip install -q datasets
import datasets
from datasets import load_metric, load_dataset

import json

In [5]:
import os 
# change dir
os.chdir("/content/drive/MyDrive/Colab Notebooks")

In [18]:
model_names = ['GPT2-xl', 'BART-large', 'BERT-large']
tokenizers_address = ['gpt2-xl', 'facebook/bart-large', "bert-large-cased"]
sparsity = [0, 0.1, 0.5, 0.9, 0.95, 0.99] 
# sparsity = [0.5] 

models = dict()
tokenizers = dict()
for model_name, tokenizer_add in zip(model_names, tokenizers_address):
  tokenizers[model_name] = tokenizer_add
  models[model_name] = []
  for prune_proportion in sparsity:
    models[model_name].append(f'{os.getcwd()}/models/{model_name}_sparsity_{prune_proportion*100}')

## Running Benchmarks

In [24]:
def run_benchmarks(models, sparsity, tokenizers, model_names):
  '''
  run benchmarks
    GLUE(qnli) and CLM(wikitext v2 raw)
  '''
  # from https://ifwind.github.io/2021/08/31/BERT实战——（7）生成任务-语言模型/#调用分词器对所有的文本分词
  # for  CLM
  def group_texts(examples):
    block_size = 256
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result
  
  # CLM
  def tokenize_function_clm(examples):
    return tokenizer(examples["text"], truncation=True)

  # GLUE
  def tokenize_function_glue(examples):
    return tokenizer(examples["question"], 
                     examples["sentence"],
                     padding="max_length",
                     max_length=256, 
                     truncation=True)
  
  # for GLUE 
  def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

  # eval_pred of bart is different
  def compute_metrics_bart(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits[0], axis=-1)
    return metric.compute(predictions=predictions, references=labels)

  for model_name in model_names:
    tokenizer = AutoTokenizer.from_pretrained(tokenizers[model_name])
    if model_name == 'GPT2-xl':
      tokenizer.pad_token = tokenizer.eos_token
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    for idx, model in enumerate(models[model_name]):
      for task in ['clm', 'glue']:
        if task == 'clm':
          dataset = datasets.load_dataset('wikitext', 'wikitext-2-raw-v1')
          tokenized_datasets = dataset.map(tokenize_function_clm, batched=True, remove_columns=["text"])
          tokenized_datasets = tokenized_datasets.map(group_texts, batched=True, batch_size=1000)
          model = AutoModelForCausalLM.from_pretrained(models[model_name][idx])
        else:
          dataset = datasets.load_dataset('glue', 'qnli')
          metric = load_metric("glue", "qnli")
          tokenized_datasets = dataset.map(tokenize_function_glue, batched=True)
          # change number of labels accordingly
          model = AutoModelForSequenceClassification.from_pretrained(models[model_name][idx], 
                                                                    num_labels=2)
        if model_name == 'GPT2-xl':
          model.config.pad_token_id = model.config.eos_token_id

        train_dataset = tokenized_datasets["train"].shuffle(seed= 42).select(range(5000))
        eval_dataset = tokenized_datasets["validation"].shuffle(seed= 42).select(range(1000))
        training_args = TrainingArguments(output_dir=f"{os.getcwd()}/training/{model_name}_{task}_sparsity_{sparsity[idx]*100}_output", 
                                    num_train_epochs=1,
                                    per_device_train_batch_size=16,
                                    per_device_eval_batch_size=32,
                                    save_strategy = "steps",
                                    save_steps = 10000,
                                    evaluation_strategy="epoch")
        
        if task == 'clm':
          trainer = Trainer(
                            model=model,
                            args=training_args,
                            train_dataset=train_dataset,
                            eval_dataset=eval_dataset,
                            data_collator=data_collator,
                            )
        else:
          if model_name == 'BART-large':
            trainer = Trainer(
                              model=model,
                              args=training_args,
                              train_dataset=train_dataset,
                              eval_dataset=eval_dataset,
                              data_collator=data_collator,
                              compute_metrics=compute_metrics_bart,
                              )
          else:
            trainer = Trainer(
                              model=model,
                              args=training_args,
                              train_dataset=train_dataset,
                              eval_dataset=eval_dataset,
                              data_collator=data_collator,
                              compute_metrics=compute_metrics,
                              )

        train_results = trainer.train()
        eval_results = trainer.evaluate()
        result = {'train': train_results.metrics,
                  'eval': eval_results}
        dir_open = open(f'{os.getcwd()}/benchmark/{model_name}_{task}_sparsity_{sparsity[idx]*100}_results.json', "w")
        json.dump(result, dir_open, indent = 4)

In [25]:
run_benchmarks(models, sparsity, tokenizers, model_names)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2-xl/snapshots/208c7068911ed684d720371bad80d9455a4492e6/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2-xl",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1600,
  "n_head": 25,
  "n_inner": null,
  "n_layer": 48,
  "n_positions": 1024,
  "output_past": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_s

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

loading configuration file /content/drive/MyDrive/Colab Notebooks/models/GPT2-xl_sparsity_50.0/config.json
Model config GPT2Config {
  "_name_or_path": "/content/drive/MyDrive/Colab Notebooks/models/GPT2-xl_sparsity_50.0",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2Model"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype

...

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

loading configuration file /content/drive/MyDrive/Colab Notebooks/models/BERT-large_sparsity_50.0/config.json
Model config BertConfig {
  "_name_or_path": "/content/drive/MyDrive/Colab Notebooks/models/BERT-large_sparsity_50.0",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading weights file /content/drive/MyDrive/Colab Notebooks/models/BERT-large_sparsity_50.0/pytorch_model.bin
All model checkpoint weights were used when 

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.659252,0.630000


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence, idx, question. If sentence, idx, question are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence, idx, question. If sentence, idx, question are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 100
  Batch size = 32


In [ ]:
!nvidia-smi

Mon Nov 14 03:51:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    48W / 400W |  40364MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!fuser -v /dev/nvidia*

                     USER        PID ACCESS COMMAND
/dev/nvidia0:        root       1092 F...m python3
/dev/nvidiactl:      root       1092 F...m python3
/dev/nvidia-uvm:     root       1092 F...m python3
